In [15]:
# Already tested and implemented the scrapping process on jupyter notebook, copy paste it here. 67k entries were scrapped
# I still need to examine the data to make sure it is up to date and no discrepancies
# Display names needs to be added, or i think it was extensionName

# extensions after the 8th are not being included. Another I could do it -> infinite scroll and gather each title name, then append it to the url and get the info. I still do not know why the below doesnt work rn
import pandas as pd
import requests
from datetime import datetime
from dateutil import parser


vscodeDataSet = pd.DataFrame(columns = ['Extension Name','Publisher','Downloads','Rating','Verified','Published','Last Update']) # I might need more cols later

url = "https://marketplace.visualstudio.com/_apis/public/gallery/extensionquery" # scrapping from this url
headers = {
        "Accept": "application/json;api-version=3.0-preview.1",
        "Content-Type": "application/json"
    }

to_add = {
    "Extension Name":[],
    "Publisher":[],
    "Downloads":[],
    "Rating":[],
    "Verified":[],
    "Published":[],
    "Last Update":[]
}
# 88 was found through trial and error. This is the maximum number of pages that can be scrapped 
for i in range(1,88):
    payload = {
            "filters": [
                {
                    "criteria": [
                        {"filterType": 8, "value": "Microsoft.VisualStudio.Code"}],
                    'pageNumber':i,
                    'pageSize':1000,
                    
                }
            ],
            "flags": 870
        }
    make_request = requests.post(url,headers=headers,json=payload)
    database = make_request.json()

    for x in range(0,1000):
        # By looking at the json file, this is how I can access each field that I want.
        name = database['results'][0]['extensions'][x]['extensionName']
        publisher = database['results'][0]['extensions'][x]['publisher']['publisherName']
        downloads = database['results'][0]['extensions'][x]['statistics'][0]['value']
        rating = database['results'][0]['extensions'][x]['statistics'][-2]['value']
        verified = database['results'][0]['extensions'][x]['publisher']['isDomainVerified']
        published = database['results'][0]['extensions'][x]['releaseDate']
        lastUpdate = database['results'][0]['extensions'][x]['lastUpdated']
        date = parser.isoparse(published)
        timestamp = int(datetime.timestamp(date))
        
        new_row = pd.DataFrame([[name,publisher,downloads,rating,verified,timestamp,lastUpdate]], columns=['Extension Name', 'Publisher', 'Downloads','Rating','Verified','Published','Last Update'])
        vscodeDataSet = pd.concat([vscodeDataSet,new_row])    
        #updated_VscodeDataSet = pd.concat([vscodeDataSet,new_row])



C:\Users\Ziyad\AppData\Local\Temp\ipykernel_26768\4173367418.py:59: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  vscodeDataSet = pd.concat([vscodeDataSet,new_row])


KeyboardInterrupt: 

In [7]:
vscodeDataSet.drop_duplicates(keep='first',inplace=True) # 1-2k duplicates found
vscodeDataSet.isnull().sum() # no values that are null
vscodeDataSet['Publisher'] == 'ms-python' # 0 values that are ms-python

0     True
0     True
0    False
0    False
0    False
     ...  
0    False
0    False
0    False
0    False
0    False
Name: Publisher, Length: 70517, dtype: bool

In [13]:
vscodeDataSet['Published'] = vscodeDataSet['Published'].sort_values(ascending=True)

In [ ]:
# It looks like extensions past the 8th of Feb are not being scrapped. Not sure if this is a problem
vscodeDataSet

,Extension Name,Publisher,Downloads,Rating,Verified,Published,Last Update
0,python,ms-python,156360018.0,4.200052,True,1444381489,2025-02-17T10:48:02.973Z
0,vscode-pylance,ms-python,128385027.0,3.094988,True,1444919480,2025-02-05T22:23:51.71Z
0,jupyter,ms-toolsai,88011011.0,2.755257,True,1444921543,2025-02-07T09:47:00.717Z
0,cpptools,ms-vscode,78590549.0,3.487278,True,1444929261,2025-02-14T00:49:35.947Z
0,jupyter-keymap,ms-toolsai,69171847.0,4.310157,True,1444929611,2023-06-05T17:53:31.993Z
...,...,...,...,...,...,...,...
0,helloworld-sample-test-np,nappa,1.0,4.403205,False,1739788281,2022-06-23T13:30:08.413Z
0,chemapack-molon,ChemaCortes,1.0,4.403205,False,1739790569,2021-01-21T01:27:17.813Z
0,yeetpoint,teemal,1.0,4.403205,False,1739800730,2023-02-27T17:20:25.103Z
0,quiz-extention,Quiz,1.0,4.457459,False,1739803392,2025-02-08T12:16:36.127Z
